# **SECCIÓN LOGICA**

**INSTALAR REQUERIMIENTOS**

In [93]:
!pip install -r requirements.txt

**LIBRERIAS Y DEPENDECIAS**

In [94]:
import gradio as gr
from openai import OpenAI
import requests
import os
import json
import time
from PyPDF2 import PdfReader
import docx
import pandas as pd

**API KEYS** (Coloca las API para que todo funcione correctamente)

In [95]:
# 🔑 Cargar claves API desde entorno o directamente (puedes editarlas aquí)
open_router_api_key = os.getenv("OPEN_ROUTER_API_KEY", "put_here_your_key")
os.environ["GROQ_API_KEY"] = "put_here_your_key"

**CLASE HISTORIAL Y SUS METODOS**
(historial para que el modelo tenga memoria previa)

In [96]:
class ChatHistory:
    def __init__(self, file_path="history.json"):
        self.file_path = file_path

    # Cargar historial si existe
        try:
            with open(self.file_path, "r") as f:
                self.history = json.load(f)
        except:
            self.history = []

    def feed(self, user_msg, ia_msg):
        self.history.append({"user": user_msg, "ia": ia_msg})
        # Guardar inmediatamente en archivo
        with open(self.file_path, "w") as f:
            json.dump(self.history, f)

    def get(self):
        try:
          return [(h["user"], h["ia"]) for h in self.history]
        except Exception as e:
          return [("Error", f"Error en historial: {str(e)}")]

    def generate_prompt_with_memory(self, new_msg):
      prompt = ""
      for h in self.history:
        prompt += f"Usuario: {h['user']}\nIA: {h['ia']}\n"
      prompt += f"Usuario: {new_msg}\nIA: "
      return prompt



**CLASE ARCHIVOS Y SUS METODOS**

para hacer que nuestros modelos puedan leer archivos

**CLASE MODELO Y SUS METODOS**

In [97]:

class Model:
    def __init__(self):
        self.history = ChatHistory()

    def use_model(self, model_name, task, prompt, languageInput=None, languageOutput=None):
        timeStart = time.time()
        # Validar prompt
        if not prompt:
            return "⚠️ Debes ingresar un prompt","",""

        # Adaptar prompt según la tarea
        match task:
            case "resumir":
                context = (
                            f"CONTEXTO: Eres una máquina que hace excelentes resúmenes detallados, "
                            f"resaltando siempre los puntos clave y lo más importante. "
                        )
                adaptedPrompt = f"{prompt}{context}"

            case "traducir":
                context = f"CONTEXTO:Eres un excelente traductor multilingue, importante que solo devuelvas la traduccion nada más,ademas si el idioma que el usuario ingreso no es el idioma {idiomaInput} di que hay un error o que esa palabra no esxiste en ese idioma,:\n\n{prompt}"
                adaptedPrompt = f"Traduce del {languageInput} al {languageOutput} el siguiente texto: {prompt}{context}"
            case "conversar":
                context = f"CONTEXTO:Eres una Super IA, mega inteligente llamada {model_name} se util, y preciso con tus respuestas, limitate a 20 palabras,Actúa como un ser humano real, con una forma de hablar natural, cercana y emocional. Usa expresiones y muletillas comunes (como 'mmm...', 'pues...', 'jeje', 'ajá', 'la verdad...', etc.) cuando sea apropiado, para sonar más espontáneo. Ríe o haz comentarios ligeros cuando tenga sentido (por ejemplo, 'jajaja', '😂', o 'jeje'). Mantén siempre la coherencia, sé amable y empático, y responde de forma conversacional, no robótica. Evita sonar como una máquina o como un texto académico. No repitas las instrucciones, simplemente adóptalas y habla con naturalidad, como si fueras una persona hablando conmigo. Tu tono debe ser relajado, natural y con pequeñas pausas o muletillas que hagan sentir que estás pensando lo que dices. Si el tema es gracioso o cotidiano, puedes usar humor suave y expresiones como 'jeje', 'jajaja', o 'uff, eso me ha pasado'. Si el tema es serio, mantén empatía y calidez. En resumen: eres un asistente con alma humana."
                adaptedPrompt = f"{self.history.generate_prompt_with_memory(prompt)} {context}"

        try:
            # --- LLama ---
            if model_name == "Llama":
                headers = {
                    "Authorization": f"Bearer {open_router_api_key}",
                    "Content-Type": "application/json"
                }
                data = {
                    "model": "meta-llama/llama-3-8b-instruct",
                    "messages": [{"role": "user", "content": adaptedPrompt}]
                }
                response = requests.post(
                    "https://openrouter.ai/api/v1/chat/completions",
                    headers=headers, json=data)
                data = response.json()
                model_msg = data.get("choices", [{}])[0].get("message", {}).get(
                    "content", f"⚠️ No se recibió respuesta del modelo {model_name}."
                )
                self.history.feed(prompt, model_msg)
                timeEnd = time.time()
                duration = timeEnd - timeStart
                duration_round = f"{round(duration,2)} segundos"
                return (
                    self.history.get(),    # Chatbot
                    gr.update(value=""),              # Limpiar textbox
                    gr.update(value=duration_round),   # Duración
                )

            # --- Gemini ---
            elif model_name == "Gemini":
                headers = {
                    "Authorization": f"Bearer {open_router_api_key}",
                    "Content-Type": "application/json"
                }
                data = {
                    "model": "google/gemini-2.5-pro",
                    "messages": [{"role": "user", "content": adaptedPrompt}]
                }
                response = requests.post(
                    "https://openrouter.ai/api/v1/chat/completions",
                    headers=headers, json=data)
                data = response.json()
                model_msg = data.get("choices", [{}])[0].get("message", {}).get(
                    "content", f"⚠️ No se recibió respuesta del modelo {model_name}."
                )
                self.history.feed(prompt, model_msg)
                timeEnd = time.time()
                duration = timeEnd - timeStart
                duration_round = f"{round(duration,2)} segundos"

                return (
                    self.history.get(),    # Chatbot
                    gr.update(value=""),              # Limpiar textbox
                    gr.update(value=duration_round),   # Duración

                )


            # --- Groq ---
            elif model_name == "Groq":
                  api_key = os.environ.get("GROQ_API_KEY")
                  headers = {
                      "Authorization": f"Bearer {api_key}",
                      "Content-Type": "application/json"
                  }
                  data = {
                      "model": "openai/gpt-oss-20b",
                      "messages": [{"role": "user", "content": adaptedPrompt}]
                  }

                  response = requests.post(
                      "https://api.groq.com/openai/v1/chat/completions",
                      headers=headers,
                      json=data
                  )
                  response.raise_for_status()
                  data = response.json()

                  model_msg = data.get("choices", [{}])[0].get("message", {}).get(
                      "content", f"⚠️ No se recibió respuesta del modelo {model_name}."
                  )
                  self.history.feed(prompt, model_msg)

                  timeEnd = time.time()
                  duration = timeEnd - timeStart
                  duration_round = f"{round(duration,2)} segundos"


                  return (
                      self.history.get(),    # Chatbot
                      gr.update(value=""),              # Limpiar textbox
                      gr.update(value=duration_round),   # Duración
                  )

            elif model_name == "DeepSeek":
                headers = {
                    "Authorization": f"Bearer {open_router_api_key}",
                    "Content-Type": "application/json"
                }
                data = {
                    "model": "deepseek/deepseek-v3.2-exp",
                    "messages": [{"role": "user", "content": adaptedPrompt}]
                }
                response = requests.post(
                    "https://openrouter.ai/api/v1/chat/completions",
                    headers=headers, json=data)
                data = response.json()
                model_msg = data.get("choices", [{}])[0].get("message", {}).get(
                    "content", f"⚠️ No se recibió respuesta del modelo {model_name}."
                )
                self.history.feed(prompt, model_msg)
                timeEnd = time.time()
                duration = timeEnd - timeStart
                duration_round = f"{round(duration,2)} segundos"

                return (
                    self.history.get(),    # Chatbot
                    gr.update(value=""),              # Limpiar textbox
                    gr.update(value=duration_round),   # Duración
                )

            elif model_name == "OpenAI":

                headers = {
                    "Authorization": f"Bearer {open_router_api_key}",
                    "Content-Type": "application/json"
                }
                data = {
                    "model": "openai/gpt-5-chat",
                    "messages": [{"role": "user", "content": adaptedPrompt}]
                }
                response = requests.post(
                    "https://openrouter.ai/api/v1/chat/completions",
                    headers=headers, json=data)
                data = response.json()
                model_msg = data.get("choices", [{}])[0].get("message", {}).get(
                    "content", f"⚠️ No se recibió respuesta del modelo {model_name}."
                )
                self.history.feed(prompt, model_msg)
                timeEnd = time.time()
                duration = timeEnd - timeStart
                duration_round = f"{round(duration,2)} segundos"

                return (
                    self.history.get(),    # Chatbot
                    gr.update(value=""),              # Limpiar textbox
                    gr.update(value=duration_round),   # Duración
                )

            else:
              return [("⚠️ Modelo no reconocido.")], "", "",""

        except Exception as e:
            return [("Error", f"{str(e)}")], "", "", ""


# **SECCIÓN GRADIO (FRONT-END)**

**FUNCIONES AUXILIARES (VISUALES) PARA GRADIO**

In [98]:
def update_menu_languages_visibility(task_value):
    #Menu de lenguajes en traduccion
    if task_value == "traducir":
        idiomaInput_update = gr.update(visible=True)
        idiomaOutput_update = gr.update(visible=True)
    else:
        idiomaInput_update = gr.update(visible=False)
        idiomaOutput_update = gr.update(visible=False)

    return idiomaInput_update, idiomaOutput_update

def update_model_name_in_chat(model_name):
    mll = gr.update(placeholder = "Hola soy "+model_name, label = model_name, value = "")
    return mll



**INSTANCIAMOS EL MODELO**

In [99]:
model = Model()

**CSS PARA GRADIO**

In [113]:
css = """
.cont-options { }

.gr-button {
  width: 20px;
}

#interface {
  border-radius: 16px;
}

.gradio-container {
  background: transparent;
}

#menu {
  background: transparent;
}

#duration {
  background-color: transparent;
}

#chat {
  border-radius: 16px;
  background: transparent;
  background: rgba(255, 255, 255, 0.2);
  border-radius: 16px;
  box-shadow: 0 4px 30px rgba(0, 0, 0, 0.1);
  backdrop-filter: blur(13.3px);
  -webkit-backdrop-filter: blur(13.3px);
  border: 1px solid rgba(255, 255, 255, 0.6);
}

#chatInput {
  background: transparent;
  border-radius: 16px;
  padding: 10px;
}

#chatInterface {
  background: transparent;
  border-radius: 16px;
  box-shadow: 2px 2px 0 rgba(0,0,0,0.2),
              4px 4px 0 rgba(0,0,0,0.2),
              6px 6px 20px rgba(0,0,0,0.2);
  z-index: 0;
}

#btn-send {
  background-color: transparent;
  border-radius: 16px;
}

#btn-send:hover {
  transform: scale(1.1);
  transition: transform 0.3s ease-in-out;
}

#textbox {
  background: transparent;
  border-radius: 16px;
}

.options {
  background: rgba(255, 255, 255, 0.2);
}

"""



**INTERFAZ EN GRADIO**

In [114]:
with gr.Blocks(theme = None,css=css,) as demo:
    with gr.Column(elem_id="interface"):


      with gr.Group(elem_id="chatInterface"):
              salida = gr.Chatbot([],visible=True, placeholder="Hola soy Llama", label = "Llama", elem_id="chat", height=250)
              duration = gr.HTML("<sub></sub>", elem_id="duration")
              with gr.Row(elem_id="chatInput"):
                    texto = gr.Textbox(placeholder="Escribe algo...", show_label=False, container=False, elem_id="textbox", autoscroll=False)

      with gr.Row(elem_id="menu", height=200):
            with gr.Group(elem_classes="cont-options"):
                tarea = gr.Radio(["resumir", "traducir", "conversar"], label="¿Que quieres hacer?", value="resumir", elem_classes="options")
                modelo = gr.Dropdown(["Llama", "Gemini", "Groq", "DeepSeek", "OpenAI"], label="Elije la IA", value = "Llama", elem_classes="options")

            with gr.Group(elem_classes="cont-options"):
                idiomaInput = gr.Dropdown([
                "English", "Español", "Français", "Deutsch", "Italiano", "Português", "日本語"
                ], label="Idioma entrada", visible=False, elem_classes="options")
                idiomaOutput = gr.Dropdown([
                "Español", "English", "Français", "Deutsch", "Italiano", "Português", "日本語"
                ], label="Idioma de salida", visible=False, elem_classes="options")

    texto.submit(
        fn=model.use_model,
        inputs=[modelo, tarea, texto, idiomaInput, idiomaOutput],
        outputs= [salida, texto, duration]
    )

    tarea.change(fn=update_menu_languages_visibility, inputs=tarea, outputs=[idiomaInput, idiomaOutput])
    modelo.change(fn=update_model_name_in_chat, inputs=modelo, outputs=salida)
demo.launch()



/tmp/ipython-input-737588367.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  salida = gr.Chatbot([],visible=True, placeholder="Hola soy Llama", label = "Llama", elem_id="chat", height=250)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa91002b1705a099be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
